# PEDS example: Maxwell surrogate

This example of PEDS shows how to use the code from: *Pestourie, Raphaël, et al. "Physics-enhanced deep surrogates for PDEs."*  In this example notebook, we illustrate PEDS for the surrogate of the diffusion equation's flux where the approximate solver is a coarse diffusion equation's solver. For this example, the input is the width of 10 layers of holes and the one-hot-encoding of three frequencies.
    
## Note on implementation

In this notebook, we run the training on a single processor and J=1.

However, the implementation of this code is meant to be parallel on multiple processors, where the element of a batch is computed in parallel over J groups of CPUs (J is the number models in the ensemble). For example, with a julia file `train_PEDS.jl`containing functions calls similart to the code in the cells below, the command to run large-scale training on a cluster would be:
    
`mpiexec -n 320 julia train_PEDS.jl` 

In [1]:
##load module
include("../src/PEDS.jl")

##loading data
X = readdlm("../data/X_maxwell10_small.csv", ',')
y = parse.(Complex{Float64}, readdlm("../data/y_maxwell10_small.csv", ',')[:])

Xv = X[:, 1:1024] #valid set
Xtest = X[:, end-1023:end] #test set
Xt = X[:, 1025:end]

yv = y[1:1024] #valid set
ytest = y[end-1023:end] #test set
yt = y[1025:end];

┌ Warning: Package cuDNN not found in current path.
│ - Run `import Pkg; Pkg.add("cuDNN")` to install the cuDNN package, then restart julia.
│ - If cuDNN is not installed, some Flux functionalities will not be available when running on the GPU.
└ @ FluxCUDAExt C:\Users\XMhua\.julia\packages\Flux\CUn7U\ext\FluxCUDAExt\FluxCUDAExt.jl:57


In [2]:
datalimitvalid = 2^10
Jval=1
batchsize=64

##Definition of problem constants
const debug = false
const drv = DataRunner(Xv, yv, [1]);
const al = ALstruct(J=Jval, Nvalid=datalimitvalid, batchsize=batchsize);
const valid = initvalid(al, drv) #validation loader

const drtest = DataRunner(Xtest, ytest, [1]);
const test = initvalid(al, drtest) #validation loader

const nn = NNstruct(outGen=[256, 256, 10*110],
postGen = [x-> @. x*1.5 + 2.5; x-> reshape(x, (110,10,:))],
inVar = [110*10, 256, 256, 256]);
const cs = CSstruct(resolution=10, 
nn_x=10, 
ny_nn=110, 
refsim=0.3364246930443735 + 0.1920021246559511im);
const sd = SimulationDomain(cs)
##setup MPI and random
const comm = MPI.COMM_WORLD
const model_color = MPI.Comm_rank(comm)%al.J
const commModel = MPI.Comm_split(comm, model_color, 0)
const isleader = MPI.Comm_rank(commModel) == 0
const commLeader = MPI.Comm_split(comm, isleader, 0)
debug && print("Comm rank=$(MPI.Comm_rank(comm)), commModel rank = $(MPI.Comm_rank(commModel)), commLeader rank = $(MPI.Comm_rank(commLeader))\n")
Random.seed!(2134*(model_color+1)) #alter seed for different groups

TaskLocalRNG()

In [3]:
##training functionalities
function train_distributed!(comm, commModel, commLeader, mloglik, m, loss, ps, loader, opt, validation_fes; logging=false)
    for d in loader
        train_loss, back = Zygote.pullback(() -> loss(commModel, mloglik, m, d...), ps)
        gs = back(1.)
        if debug && isleader
            if isnan(train_loss)
                @show (model_color, train_loss)
            end
        end
        for x in ps
            gs[x][:] .= sum_reduce(commModel, Float64.(gs[x][:]))
            if debug && isleader
                if any(isnan.(gs[x][:]))
                    @show (model_color, train_loss)
                    @show (length(x), length(findall(isnan,x)), length(findall(isnan,gs[x][:])))
                end

                if any(isinf.(gs[x][:]))
                    @show (model_color, train_loss)
                    @show (length(x), length(findall(isinf,x)), length(findall(isinf,gs[x][:])))
                end
            end
        end
        
        if debug && isleader
            for p_ in ps
                if any(isnan.(p_))
                    @show (model_color, "before update")
                end
            end
        end
        Flux.Optimise.update!(opt, ps, gs)
        if debug && isleader
            for p_ in ps
                if any(isnan.(p_))
                    @show (model_color, "after update")
                end
            end
        end
    end

    logging && push!(validation_fes, dFE(comm, commModel, commLeader, m))
end

function dFE(comm, commModel, commLeader, m; valid=valid) """dFE computes the FE using parallelization over the batch with MPI""" 
    evalsr = zeros(al.Nvalid)
    evalsi = zeros(al.Nvalid)
    FE = 0.
    j=0
    ys = Complex{Float64}[]
    for (x, y) in valid
        for i=1+MPI.Comm_rank(commModel):MPI.Comm_size(commModel):length(y)
            rp, ip = m(x[:,i])
            evalsr[j*length(y)+i] = rp
            evalsi[j*length(y)+i] = ip
        end
        j+=1
        push!(ys, y...)
    end
    evalsrModel = sum_reduce(commModel, evalsr)
    evalsiModel = sum_reduce(commModel, evalsi)
    evalsr = sum_reduce(commLeader, evalsrModel) / al.J
    evalsi = sum_reduce(commLeader, evalsiModel) / al.J
    if MPI.Comm_rank(comm) == 0
        ŷ = @. evalsr + 1im * evalsi
        FE = norm(ŷ - ys)/norm(ys)
        @show FE
    end
    return FE
end

dFE (generic function with 1 method)

## PEDS

In [4]:
kval=2^7

##define same AL parameters for all workers
MPI.Barrier(comm)
al1 = ALstruct(Ninit=256+8*kval, T=0);
if MPI.Comm_rank(comm) == 0
    @show kval
end
##ititialize DataRunner and DataSet
dr = DataRunner(Xt, yt, [1]);
ds = DataSet()
validation_fes = []
##initialize baseline
(mgen, cw, mvar) = initmodel(nn)
coarseinput(p) = begin 
    (coarsified, sd_freq) = Zygote.ignore() do
        coarse_geom_func(p)
    end
    generated =dropdims(mgen(p), dims=3)
    debug && isleader && any(isnan.(ϵcombine)) && writedlm("inputnan", p) 
    debug && isleader && any(isnan.(ϵcombine)) && writedlm("mgennan", mgen(p)) 
    debug && isleader && any(isnan.(ϵcombine)) && writedlm("mgennanparam", ps)
    debug && isleader && any(isnan.(ϵcombine)) && writedlm("errorcolor", model_color)
    w = NNlib.sigmoid.(cw*nn.multfact)
    # w = max(0, min(1, cw))
    ϵcombine = @. w * generated + (1-w) * coarsified
    ϵsymmetric = ϵcombine#(ϵcombine .+ reverse(ϵcombine, dims=2))./2
    return ϵsymmetric, sd_freq
end
m(p) = begin
    ϵsymmetric, sd_freq = coarseinput(p)
    return [realtransmissionSolver(ϵsymmetric, sd_freq = sd_freq); imagtransmissionSolver(ϵsymmetric, sd_freq = sd_freq)]
end 

uq(p) = mvar(coarseinput(p)[1])

mloglik(p) =  vcat(m(p), uq(p))
ps = Flux.params(mgen, cw, mvar)
# ps = Flux.params(mgen, mvar)
loader = initloader(al1, dr, ds);
opt = ADAM(al1.η)
##train baseline
if MPI.Comm_rank(comm)==0
    @time for iepoch = 1:al1.ne 
        train_distributed!(comm, commModel, commLeader, mloglik, m, dNLL, ps, loader, opt, validation_fes, logging=true)
    end
else
    for iepoch = 1:al1.ne 
        train_distributed!(comm, commModel, commLeader, mloglik, m, dNLL, ps, loader, opt, validation_fes, logging=true)
    end
end
##active learning loop
for t=1:al1.T
    MPI.Comm_rank(comm) == 0 && @show t
    loader= getloader(al1, dr, ds, X->varfilter(mloglik, X))
    if MPI.Comm_rank(comm)==0 
        @time for iepoch = 1:al1.ne 
            train_distributed!(comm, commModel, commLeader, mloglik, m, dNLL, ps, loader, opt, validation_fes, logging=true)
        end
    else
        for iepoch = 1:al1.ne 
            train_distributed!(comm, commModel, commLeader, mloglik, m, dNLL, ps, loader, opt, validation_fes, logging=true)
        end
    end
end

# ##save models and validation FEs
# name = "PEDS10_example"
# if isleader
#     BSON.@save "$(name)_K$(kval)_mgen$(model_color).bson" mgen
#     BSON.@save "$(name)_K$(kval)_cw$(model_color).bson" cw
#     BSON.@save "$(name)_K$(kval)_mvar$(model_color).bson" mvar
# end
# if MPI.Comm_rank(comm) == 0
#     writedlm("$(name)_K$(kval)_validation_fes.csv", validation_fes, ',')
# end

kval = 128


┌ Warning: Layer with Float32 parameters got Float64 input.
│   The input will be converted, but any earlier layers may be very slow.
│   layer = Dense(13 => 256, relu)  # 3_584 parameters
│   summary(x) = "13-element Vector{Float64}"
└ @ Flux C:\Users\XMhua\.julia\packages\Flux\CUn7U\src\layers\stateless.jl:60


FE = 1.3602127809819944
FE = 1.3335138053311935
FE = 0.9220091263809985
FE = 0.8535235023585223
FE = 0.7912842866398608
FE = 0.46707727421604744
FE = 0.429364013652819
FE = 0.34384514757418494
FE = 0.3007922969038417
FE = 0.31037220996540593
353.417406 seconds (76.69 M allocations: 418.373 GiB, 5.54% gc time, 6.87% compilation time)


In [5]:
dFE(comm, commModel, commLeader, m, valid=test)

FE = 0.318472620296603


0.318472620296603

## PEDS result

The fractional error on the test set is 0.278.

## Baseline

In [6]:
##define same AL parameters for all workers
MPI.Barrier(comm)
al1 = ALstruct(Ninit=256+8*kval, T=0);

if MPI.Comm_rank(comm) == 0
    @show kval
end
##ititialize DataRunner and DataSet
dr = DataRunner(Xt, yt, [1]);
ds = DataSet()
validation_fes = []
##initialize baseline
(mgen, pred, mvar) = initbase(nn)
mloglik(p) =  vcat(pred(mgen(p)), mvar(mgen(p)))
m(p) = pred(mgen(p))
ps = Flux.params(mgen, pred, mvar)
loader = initloader(al1, dr, ds);
opt = ADAM(al1.η)
##train baseline
if MPI.Comm_rank(comm)==0
    @time for iepoch = 1:al1.ne 
        train_distributed!(comm, commModel, commLeader, mloglik, m, dNLL, ps, loader, opt, validation_fes, logging=true)
    end
else
    for iepoch = 1:al1.ne 
        train_distributed!(comm, commModel, commLeader, mloglik, m, dNLL, ps, loader, opt, validation_fes, logging=true)
    end
end
##active learning loop
for t=1:al1.T
    MPI.Comm_rank(comm) == 0 && @show t
    loader= getloader(al1, dr, ds, X->varfilter(mloglik, X)) 
    if MPI.Comm_rank(comm)==0 
        @time for iepoch = 1:al1.ne 
            train_distributed!(comm, commModel, commLeader, mloglik, m, dNLL, ps, loader, opt, validation_fes, logging=true)
        end
    else
        for iepoch = 1:al1.ne 
            train_distributed!(comm, commModel, commLeader, mloglik, m, dNLL, ps, loader, opt, validation_fes, logging=true)
        end
    end
end

# ##save models and validation FEs
# name = "baseline10_noal_example"
# if isleader
#     BSON.@save "$(name)_K$(kval)_mgen$(model_color).bson" mgen
#     BSON.@save "$(name)_K$(kval)_pred$(model_color).bson" pred
#     BSON.@save "$(name)_K$(kval)_mvar$(model_color).bson" mvar
# end
# if MPI.Comm_rank(comm) == 0
#     writedlm("$(name)_K$(kval)_validation_fes.csv", validation_fes, ',')
# end

kval = 128
FE = 0.8060907234972654
FE = 0.7383732494364024
FE = 0.6204775997590223
FE = 0.5923675026062127
FE = 0.5361577078931435
FE = 0.539619721796218
FE = 0.5302943719601326
FE = 0.5456588695261273
FE = 0.49424089719077735
FE = 0.4902350352298319
 22.341325 seconds (9.78 M allocations: 70.384 GiB, 24.18% gc time, 6.42% compilation time: 19% of which was recompilation)


In [7]:
dFE(comm, commModel, commLeader, m, valid=test)

FE = 0.5027021042100664


0.5027021042100664

## Baseline result 

The fractional error on the test set is 0.541.

# Overall result

With about 1000 data points, a single model PEDS model leads to a 1.9x improvement compared to the NN-only baseline. For the Maxwell surrogate, in contrast to diffusion and reaction-diffusion equations, more points are needed to achieve an accuracy comparable to fabrication error (< 5%).